## Struktury danych

In [6]:
class Point:
    def __init__(self,x,y):
        self.x=x
        self.y=y

In [7]:
class Segment:
    def __init__(self,p,q):
        
        if p.x < q.x:
            left, right=p, q
        else:
            left, right=q, p
                    
        self.left=left
        self.right=right
        self.a=(right.y-left.y)/(right.x-left.x) # zakladam ze nie ma pionowych odcinkow
        self.b=left.y-left.x*self.a


In [8]:
class Trapezoid:
    def __init__(self, top, bottom, leftp, rightp):

        self.top = top
        self.bottom = bottom
        self.leftp = leftp
        self.rightp = rightp

        # wskazniki do sasiadow
        self.upper_left = None
        self.lower_left = None
        self.upper_right = None
        self.lower_right = None


In [3]:
class DNode:
    def __init__(self, node_type, label, left=None, right=None):
        """
        node_type: Typ węzła ('x', 'y' lub 'leaf')
        label: Punkt, odcinek lub trapez, zależnie od typu węzła
        left: Lewy potomek
        right: Prawy potomek
        """
        self.node_type = node_type  # Typ węzła: 'x', 'y', 'leaf'
        self.label = label          # Punkt, odcinek lub trapez
        self.left = left            # Lewy potomek
        self.right = right          # Prawy potomek

class DTree:
    def __init__(self):
        self.root = None

    def query(self, point):
        """
        Znajduje trapez, który zawiera dany punkt.
        point: (x, y) - współrzędne punktu
        Zwraca: Trapez (etykieta liścia), który zawiera punkt
        """
        current = self.root
        while current:
            if current.node_type == 'x':
                # Testujemy po współrzędnej x
                if point.x < current.label.x:
                    current = current.left
                else:
                    current = current.right

            elif current.node_type == 'y':
                # Testujemy położenie względem odcinka
                segment = current.label
                y_on_line = segment.a * point.x + segment.b
                if point.y > y_on_line:
                    current = current.left
                else:
                    current = current.right

            elif current.node_type == 'leaf':
                # Znaleziony trapez
                return current.label

        return None

# Na wykładzie są podane dwa rodzaje tej struktury - musimy wytłumaczyć dlaczego akurat tą wybierami?

In [9]:
# Testy struktury DTree
if __name__ == "__main__":
    # Tworzenie przykładowych danych
    p1 = Point(1, 1)
    p2 = Point(5, 5)
    p3 = Point(3, 2)
    segment = Segment(p1, p2)

    trapez1 = Trapezoid(None, None, p1, p2)
    trapez2 = Trapezoid(None, None, p2, p3)

    leaf1 = DNode('leaf', trapez1)
    leaf2 = DNode('leaf', trapez2)
    x_node = DNode('x', p2, leaf1, leaf2)

    # Tworzenie drzewa
    dtree = DTree()
    dtree.root = x_node

    # Test punktu
    test_point = Point(4, 4)
    result = dtree.query(test_point)
    print("Trapez zawierający punkt:", result.leftp.x, result.rightp.x)

Trapez zawierający punkt: 1 5
